In [ ]:
# ─────────────────────────────────────────────
# Cell 1: Install dependencies
# ─────────────────────────────────────────────
!pip install ultralytics --quiet
!pip install matplotlib --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.8 MB/s eta 0:00:00


In [2]:
# ─────────────────────────────────────────────
# Cell 1: Mount Drive & Prepare Paths
# ─────────────────────────────────────────────
from google.colab import drive
drive.mount('/content/drive')

# 1. Where you permanently saved the YOLO‑ready dataset
YOLO_READY_DIR = "/content/drive/MyDrive/thermo_yolo/yolo_ready_dataset"


Mounted at /content/drive


In [7]:
# 2. (Optional but recommended) Copy to local for fastest I/O
LOCAL_DIR = "/content/yolo_ready_dataset"
import os, shutil

if not os.path.exists(LOCAL_DIR):
    print("📦 Copying YOLO‑ready dataset to local disk (one‑time)...")
    shutil.copytree(YOLO_READY_DIR, LOCAL_DIR)
else:
    print("✅ Local copy exists, using that.")


📦 Copying YOLO‑ready dataset to local disk (one‑time)...


KeyboardInterrupt: 

In [8]:
# 3. Point at the dataset YAML
DATA_YAML = f"{LOCAL_DIR}/thermal_dataset.yaml"
print("🔍 Using dataset YAML at:", DATA_YAML)

🔍 Using dataset YAML at: /content/yolo_ready_dataset/thermal_dataset.yaml


In [9]:
# ─────────────────────────────────────────────
# Cell 2: Install & Import Ultralytics
# ─────────────────────────────────────────────


from ultralytics import YOLO
import torch

device = 0 if torch.cuda.is_available() else 'cpu'
print("🚀 Training on device:", device)


🚀 Training on device: cpu


In [ ]:
yaml_path = "/content/yolo_ready_dataset/thermal_dataset.yaml"

with open(yaml_path, "w") as f:
    f.write("""\
path: /content/yolo_ready_dataset
train: images/train
val: images/val
nc: 2
names:
  0: Human
  1: Dog
""")

print("✅ YAML updated.")


✅ YAML updated.


In [ ]:
# ─────────────────────────────────────────────
# Cell 3: Train YOLOv8 (with early stopping)
# ─────────────────────────────────────────────
model = YOLO("yolov8n.pt")  # or yolov8s.pt, etc.

results = model.train(
    data=DATA_YAML,
    epochs=30,
    patience=6,                 # stop after 6 epochs w/o improvement
    imgsz=416,
    batch=8,
    device=device,
    workers=8,
    cache="disk",                 # ram if on gpu
    half=True,
    augment=False,
    project="/content/drive/MyDrive/thermo_yolo/runs",
    name="train1",
    exist_ok=True
)


Ultralytics 8.3.155 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (AMD EPYC 7B12)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_ready_dataset/thermal_dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=6, perspective=0.0, plots=True, pose=12.0, pret

train: Scanning /content/yolo_ready_dataset/labels/train.cache... 5068 images, 52 backgrounds, 0 corrupt: 100%|██████████| 5072/5072 [00:00<?, ?it/s]
train: Caching images (17.4GB Disk): 100%|██████████| 5072/5072 [02:40<00:00, 31.63it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2080.8±533.7 MB/s, size: 72.3 KB)



val: Scanning /content/yolo_ready_dataset/labels/val.cache... 1268 images, 12 backgrounds, 0 corrupt: 100%|██████████| 1268/1268 [00:00<?, ?it/s]
val: Caching images (4.4GB Disk): 100%|██████████| 1268/1268 [00:44<00:00, 28.70it/s]

Plotting labels to /content/drive/MyDrive/thermo_yolo/runs/train1/labels.jpg... 


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      27/30         0G      1.185      0.506     0.8681         12        416: 100%|██████████| 634/634 [21:26<00:00,  2.03s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 80/80 [01:58<00:00,  1.48s/it]

                   all       1268       1883      0.978      0.972      0.991       0.67



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      28/30         0G      1.174     0.5028     0.8615         11        416:  83%|████████▎ | 528/634 [17:58<03:39,  2.07s/it]

In [3]:
!ls -lh /content/drive/MyDrive/thermo_yolo/runs/train1/weights/


total 36M
-rw------- 1 root root 18M Jun 14 22:35 best.pt
-rw------- 1 root root 18M Jun 14 22:35 last.pt


In [ ]:
# ─────────────────────────────────────────────
# Cell 4: Plot Training Curves (from Drive)
# ─────────────────────────────────────────────
from ultralytics.utils.plotting import plot_results
import matplotlib.pyplot as plt

csv_path = "/content/drive/MyDrive/thermo_yolo/runs/train1/results.csv"
plot_results(file=csv_path)

# Display the saved plot image
img = plt.imread("/content/drive/MyDrive/thermo_yolo/runs/train1/results.png")
plt.figure(figsize=(10,5))
plt.imshow(img)
plt.axis('off')


In [ ]:
# ─────────────────────────────────────────────
# Cell 5: Evaluate & Sample Inference
# ─────────────────────────────────────────────
from pathlib import Path
import matplotlib.pyplot as plt

# Load your best weights
best_weights = "/content/drive/MyDrive/thermo_yolo/runs/train1/weights/best.pt"
model = YOLO(best_weights)

# 1) Validate on val set (uses the same YAML)
val_metrics = model.val(data=DATA_YAML)
print("Validation metrics:", val_metrics.metrics)

# 2) Plot a couple of sample predictions
VAL_IMAGES = Path(f"{LOCAL_DIR}/images/val")
for img_path in list(VAL_IMAGES.glob("*.jpg"))[:3]:
    results = model(img_path)
    annotated = results[0].plot()
    plt.figure(figsize=(8,6))
    plt.imshow(annotated)
    plt.title(img_path.name)
    plt.axis('off')


In [ ]:
import time
print("Still alive at", time.strftime("%X"))
